In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
cnx = mysql.connector.connect(user='root', password='taskbase',
                              host='127.0.0.1', port='3309', database='lernnavi')

In [4]:
query = "SELECT * FROM TaskTypes"
df_types = pd.read_sql(query, cnx)
df_types.rename(columns={'id':'typeId', 'name':'type'}, inplace=True)
df_types.head()

,typeId,type
0,26,BIT_MATCH
1,32,BIT_MATCH_MATRIX
2,27,BIT_MATCH_PICTURE
3,30,BIT_MULTIPLE_CHOICE
4,31,BIT_MULTIPLE_RESPONSE


In [5]:
query = "SELECT * FROM Tasks"
df_tasks = pd.read_sql(query, cnx)
df_tasks = pd.merge(df_tasks, df_types, how="inner", on='typeId')
df_tasks.head()

,version,taskId,title,typeId,createdTime,authorId,content,Status,versionComment,aiStatus,feedbackComment,customerId,language,followUpTask,type
0,8,Dyx07vCUuWqWSiHC6V,,3,1531728113426,2,"{""id"": ""Dyx07vCUuWqWSiHC6V"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
1,9,Dyx07vCUuWqWSiHC6V,Aufgabe PT1.2,3,1531728228715,2,"{""id"": ""Dyx07vCUuWqWSiHC6V"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
2,20,HurIW8u0XJAWSiN5aR,,3,1531729909418,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
3,21,HurIW8u0XJAWSiN5aR,,3,1531730025414,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT
4,24,HurIW8u0XJAWSiN5aR,Aufgabe PT1.6a,3,1531730573408,2,"{""id"": ""HurIW8u0XJAWSiN5aR"", ""type"": ""HIGHLIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT


In [6]:
query = "SELECT * FROM LatestTaskVersions"
df = pd.read_sql(query, cnx)
df_latest_task_versions = pd.merge(df_tasks, df, how="inner", on=["version", "taskId"])
df_latest_task_versions.head()

,version,taskId,title,typeId,createdTime_x,authorId,content,Status,versionComment,aiStatus,feedbackComment,customerId,language,followUpTask,type,id,createdTime_y
0,2325,PT1-11a-restored,Aufgabe PT1.11a,3,1535643328825,2,"{""id"": ""PT1-11a-restored"", ""type"": ""HIGHLIGHT""...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,2693,1534434704301
1,2388,7kQh-veLQfOanjx1QkhbKx,Aufgabe PT 2.14a,3,1535660365797,2,"{""id"": ""7kQh-veLQfOanjx1QkhbKx"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,1589,1532335779747
2,2395,d7SwsoVyAUP80fMioyhxzH,Aufgabe PT 2.14c,3,1535661392063,2,"{""id"": ""d7SwsoVyAUP80fMioyhxzH"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,2516,1532336772750
3,3786,4R58yTMxw8b5CK1F72eh0W,Musteraufgabe D_Markieren_Adjektive_3.4_WORT_A_1,3,1562516756370,32,"{""id"": ""4R58yTMxw8b5CK1F72eh0W"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,927,1559076209311
4,3788,2UiRhzBKL217r4pFiCA6Ek,Ersatz 1.6b_Markieren mit mehreren Farben,3,1562516867617,32,"{""id"": ""2UiRhzBKL217r4pFiCA6Ek"", ""type"": ""HIGH...",WORK_IN_PROGRESS,None,INITIAL,None,None,None,None,HIGHLIGHT,425,1555145282057


In [7]:
columns_to_keep = ["taskId", "type", "title", "language"]
df = df_latest_task_versions[columns_to_keep]

In [8]:
df_content = df_latest_task_versions["content"].apply(json.loads).apply(pd.Series)
df_content.rename(columns={"id":"taskId"}, inplace=True)

In [9]:
df_open_tasks = df_content[df_content["type"] == "OPEN"].dropna(axis=1, how="all")
columns_to_keep = ["taskId", "tenant", "language", "title", "description", "context", "topic", "hints", "solutionSteps", "replacements", "resource", "sampleSolutions", "stepifier"]
df_open_tasks = df_open_tasks[columns_to_keep]
df_open_tasks.head()

,taskId,tenant,language,title,description,context,topic,hints,solutionSteps,replacements,resource,sampleSolutions,stepifier
14687,4HNL62l.47Ta0sHqrOzKoF,NaN,NaN,Fragebogen SG 10,<p>Was hat dir besonders gut gefallen? </p>,NaN,NaN,[],[],NaN,NaN,[],NaN
14688,aCF0hfb1kQEayDOqwEp0s1,NaN,NaN,Fragebogen SG 11,<p>Was k&ouml;nnte verbessert werden?</p>,NaN,NaN,[],[],NaN,NaN,[],NaN
14689,dCZ4cWixQrM9ovYwc-TP.z,NaN,NaN,Fragebogen SG 13,<p>Welche wichtigen Features fehlen in unserer...,NaN,NaN,[],[],NaN,NaN,[],NaN
14690,8iXTTKYbQ.Hbd8daUV8dcQ,NaN,NaN,PH nach 2,<p>Was hat Ihnen besonders gut gefallen?</p>,NaN,NaN,[],[],NaN,NaN,[],NaN
14691,eEFZ3WW6kHx9rb9tdtx0AZ,NaN,NaN,PH nach 3,<p>Was k&ouml;nnte verbessert werden?</p>,NaN,NaN,[],[],NaN,NaN,[],NaN


In [10]:
df_open_tasks.dropna(subset=["tenant"], inplace=True)

In [11]:
df_open_tasks["tenant"] = df_open_tasks["tenant"].apply(pd.Series)["name"]

In [15]:
df_open_tasks = df_open_tasks[df_open_tasks["tenant"].isin(["Cornelsen DE", "Cornelsen EN", "EKV_Physik"]) ]

**Cleaning description**

In [16]:
df_open_tasks["description"] = df_open_tasks["description"].apply(lambda str: BeautifulSoup(str, 'html.parser').get_text().strip())
df_open_tasks.head()

,taskId,tenant,language,title,description,context,topic,hints,solutionSteps,replacements,resource,sampleSolutions,stepifier
15817,awg5HdhPj3M8BT32YCOHDF,Cornelsen EN,DE,"(API TEST, please ignore) 3.2 Gedicht | Inhalt...",,NaN,NaN,[],[],NaN,NaN,[],SENTENCE
15827,81xdUMIFiyC7gTSwml1MTv,Cornelsen DE,DE,Archive | 4.1 Erlebnislyrik | Kennzeichen,Was ist ein Kennzeichen der Erlebnislyrik? Ant...,NaN,"{'id': 5350, 'name': 'Essay 3', 'description':...",[],[],NaN,NaN,[],KEYWORD
15828,7fE9sk0x2amalma0takBp5,Cornelsen DE,DE,Archive | 3.3 Schlussteil Interpretationsaufsatz,Welches Vorgehen planen Sie für den Schlusstei...,NaN,"{'id': 5349, 'name': 'Essay 1', 'description':...",[],[],NaN,NaN,[],KEYWORD
15829,7CUm7BcYBG79uI8VPMUKCo,Cornelsen DE,DE,Archive | 3.2 Gedicht | Inhaltliche Aspekte,Auf welchen inhaltlichen Aspekt hin würden Sie...,NaN,"{'id': 5349, 'name': 'Essay 1', 'description':...",[],[],NaN,NaN,[],KEYWORD
15869,6eVh0sUk73laFRNimbk7MC,Cornelsen DE,DE,DaF | Relativpronomen,Mache aus den zwei Sätzen einen Relativsatz.Ic...,NaN,"{'id': 5357, 'name': 'DaF', 'description': '',...",[],"[Ich arbeite gern mit dem Laptop, den ich letz...",NaN,NaN,[],KEYWORD


In [20]:
df_open_tasks.dropna(subset=["description"], inplace=True)
df_open_tasks = df_open_tasks[df_open_tasks["description"] != ""]

In [22]:
df_open_tasks["word_count"] = df_open_tasks["description"].apply(lambda s: len(s.split()))

In [25]:
df_open_tasks.to_csv("data/open_tasks_Corn_EKV.csv", index_label=False)
df_open_tasks[df_open_tasks["language"] == "EN"].to_csv("data/open_tasks_Corn_EKV_EN.csv", index_label=False)
df_open_tasks[df_open_tasks["language"] == "DE"].to_csv("data/open_tasks_Corn_EKV_DE.csv", index_label=False)

**Fixing language field**